In [1]:
import numpy as np
import itertools
import os
import pyslha

In [2]:
mHSCP = np.linspace(25.,3000.,100)

In [3]:
# columns = ('msl3','Wsl3')
columns = ['msl3']
outputFile = 'THSCPM1b_pts_validation.txt'
header = '%19s'*len(columns) %tuple(columns)
header = header[3:]
np.savetxt(outputFile,mHSCP,header=header,fmt = ['     %1.7e']*len(columns))

In [4]:
widths = np.logspace(-25,-15,30)

pts = list(itertools.product(mHSCP,widths))
template = 'THSCPM1b_template.slha'
slhaData = pyslha.readSLHAFile(template)
for mass,width in pts: 
    newFile = './slha/THSCPM1b_%1.0f_%1.2e.slha' %(mass,width)
    slhaData.blocks['MASS'][1000015] = mass
    slhaData.decays[1000015].totalwidth = width
    with open(newFile,'w') as f:
        f.write(pyslha.writeSLHA(slhaData,ignorenobr=True,precision=5))
